In [ ]:
import numpy as np
import plotly.express as px
from rindti.models import NoisyNodesClassModel, ClassificationModel, InfoGraphModel, GraphLogModel, PfamModel, BGRLModel
from rindti.models.bgrl import Encoder
from rindti.utils.data import Dataset
import pickle
import pandas as pd
import seaborn as sns
from sklearn.manifold import TSNE
from copy import deepcopy, copy
from rindti.utils.data import PreTrainDataset
from torch_geometric.data import DataLoader
import torch
from rindti.utils.transforms import mask_features

In [ ]:
ds = pd.read_pickle("/scratch/SCRATCH_NVME/ilya/pretrain_data/pfam_1000.pkl")
ds['fam'] = ds['data'].apply(lambda x: x['fam'].split(";")[0])

In [ ]:
vc = ds['fam'].value_counts()

sub = ds[ds['fam'].isin(vc.head(10).index)].sample(2000)

In [ ]:
def embed(encoder, data):
    data['batch'] = torch.zeros_like(data['x'])
    return encoder(data).detach().numpy().reshape(-1)

In [ ]:
model = PfamModel.load_from_checkpoint('tb_logs/pfam/version_1/checkpoints/epoch=320-step=878255.ckpt')
model.eval()
encoder = model.encoder
encoder.return_nodes = False

In [ ]:
embeds = {}
for id, orig_prot in sub['data'].to_dict().items():
    prot = copy(orig_prot)
    emb = embed(encoder, prot)
    embeds[id] = emb
prot_embeds = pd.DataFrame(embeds).T

In [ ]:
tsne = TSNE()
x = tsne.fit_transform(prot_embeds)
x = pd.DataFrame(index=sub.index, data=x).join(sub)

In [ ]:
fig = px.scatter(x, 0, 1, opacity=0.7, color="fam", width=1000, height=800, hover_name=x.index)
# fig.update_traces(marker=dict(size=5))
fig.show()

In [ ]:
sns.clustermap(prot_embeds)

In [ ]:
from workflow.scripts.prepare_proteins import node_encoding

In [ ]:
embs = {}
for k,v in node_encoding.items():
    embs[k] = encoder.feat_embed(torch.tensor(v)).detach().numpy().reshape(-1)

In [ ]:
embs = pd.DataFrame(embs).T

In [ ]:
pca = PCA()
x = pca.fit_transform(embs)
x = pd.DataFrame(data=x, index=embs.index)

In [ ]:
from sklearn.decomposition import PCA

In [ ]:
px.scatter(x, 0, 1, hover_name=x.index, text=x.index, opacity=0)

In [ ]:
model.hparams.hidden_dim